In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set page config
st.set_page_config(page_title="Stock Price Prediction", layout="wide")

# Title and description
st.title("Stock Price Prediction App")
st.markdown("""
This app allows you to:
- Upload a dataset containing stock price data.
- Train machine learning models (Logistic Regression, Naive Bayes, Random Forest).
- Evaluate and compare the performance of these models.
- Predict stock price movements (e.g., Up or Down).
""")

# Sidebar for dataset upload
st.sidebar.header("Dataset Upload")
uploaded_file = st.sidebar.file_uploader("Upload your CSV file", type="csv")

if uploaded_file:
    # Load dataset
    df = pd.read_csv(uploaded_file)
    st.write("### Dataset Preview")
    st.dataframe(df.head())

    # Sidebar for feature selection
    st.sidebar.header("Feature Selection")
    target_column = st.sidebar.selectbox("Select the target column", options=df.columns)
    feature_columns = st.sidebar.multiselect("Select feature columns", options=[col for col in df.columns if col != target_column])

    if target_column and feature_columns:
        # Preprocess data
        X = df[feature_columns]
        y = df[target_column]

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Sidebar for model selection
        st.sidebar.header("Model Selection")
        models = {
            "Logistic Regression": LogisticRegression(max_iter=1000),
            "Naive Bayes": GaussianNB(),
            "Random Forest": RandomForestClassifier(random_state=42)
        }
        selected_model_name = st.sidebar.selectbox("Select a model", options=list(models.keys()))

        if selected_model_name:
            model = models[selected_model_name]

            # Train the model
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Display evaluation metrics
            st.write(f"### Model: {selected_model_name}")
            st.write(f"**Accuracy:** {accuracy:.2f}")
            st.write("### Confusion Matrix")
            st.write(confusion_matrix(y_test, y_pred))
            st.write("### Classification Report")
            st.text(classification_report(y_test, y_pred))

            # Prediction section
            st.write("### Make Predictions")
            prediction_input = st.text_area("Enter feature values for prediction (comma-separated)")

            if prediction_input:
                try:
                    input_values = np.array([float(x) for x in prediction_input.split(",")]).reshape(1, -1)
                    prediction = model.predict(input_values)
                    st.write(f"**Predicted Class:** {prediction[0]}")
                except Exception as e:
                    st.error(f"Invalid input: {e}")

else:
    st.info("Please upload a CSV file to get started.")
